In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

from collections import defaultdict
import pandas as pd
import numpy as np

from WA_state_API_functions import get_committee_member_data

# Load

In [2]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')

In [3]:
vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)

In [4]:
# bill_df = pd.read_sql_query('select * from "bill_api"',con=engine)

In [5]:
# bill_df['bill_unique'] = bill_df['bill_id'] + ' ' + bill_df['biennium']

In [6]:
# len(bill_df['bill_unique'].unique())

In [7]:
# bill_df.head()

In [8]:
vote_df['bill_unique'] = vote_df['bill_id'] + ' ' + vote_df['biennium']

In [9]:
len(vote_df['bill_unique'].unique())

19190

In [10]:
len(vote_df['voter_id'].unique())

530

In [11]:
committee_member_df = pd.read_sql_query('select * from "committee_member_api"',con=engine)

In [12]:
committee_member_df.head()

,acronym,agency,biennium,committee_name,district,first_name,id,last_name,long_name,name,party
0,ANDE,Senate,1993-94,Government Operations,42,Ann,6,Anderson,Senator Anderson,Ann Anderson,Republican
1,PELE,House,1995-96,Appropriations,25,Grant,1657,Pelesky,Representative Pelesky,Grant Pelesky,Republican
2,ELLI,House,1995-96,Education,1,Ian,1608,Elliot,Representative Elliot,Ian Elliot,Republican
3,MORR,House,1995-96,Finance,18,Betty,294,Morris,Representative Morris,Betty Morris,Democrat
4,HYME,House,1995-96,Health Care,40,Cheryl,1670,Hymes,Representative Hymes,Cheryl Hymes,Republican


# Start legislators_df

In [13]:
legislators_df = vote_df.loc[:, ['biennium', 'voter_id', 'voter_name', 'voting_agency']]

In [14]:
legislators_df.drop_duplicates(keep='first', inplace=True)

In [15]:
legislators_df.head()

,biennium,voter_id,voter_name,voting_agency
0,1991-92,7,Anderson,House
1,1991-92,11,Appelwick,House
2,1991-92,17,Ballard,House
3,1991-92,23,Basich,House
4,1991-92,32,Beck,House


In [16]:
committee_member_df_short = committee_member_df.loc[:, ['district', 'id', 'first_name', 'party']]

In [17]:
committee_member_df_short.columns = ['c_district', 'c_id', 'c_first_name', 'c_party']

In [18]:
committee_member_df_short['c_first_name'] = committee_member_df_short['c_first_name'].apply(lambda x: x.split()[0])

In [19]:
committee_member_df_short.drop_duplicates(keep='first', inplace=True)

In [43]:
legislators_df_merged = legislators_df.merge(committee_member_df_short, how='outer', left_on='voter_id', right_on='c_id')

In [44]:
legislators_df_merged.drop_duplicates(keep="first", inplace=True)

In [78]:
nulls = legislators_df_merged[legislators_df_merged['c_first_name'].isnull()]

In [89]:
len(nulls['voter_id'].unique())

156

In [108]:
nulls[nulls['voter_name'] == 'Myers']

,biennium,voter_id,voter_name,voting_agency,c_district,c_id,c_first_name,c_party


In [109]:
manual = [{'name': 'Ann Anderson', 'voter_id': '7', 'district': '42', 'party': 'R'},
 {'name': 'Bob Basich', 'voter_id': '23', 'district': '19', 'party': 'D'},
 {'name': 'John Beck', 'voter_id': '32', 'district': '21', 'party': 'R'},
 {'name': 'Jennifer Belcher', 'voter_id': '34', 'district': '22', 'party': 'D'},
 {'name': 'John Betrozoff', 'voter_id': '40', 'district': '45', 'party': 'R'},
 {'name': 'Rose Bowman', 'voter_id': '45', 'district': '20', 'party': 'R'},
 {'name': 'Dennis Braddock', 'voter_id': '46',  'district': '42', 'party': 'D'},
 {'name': 'Lane Bray', 'voter_id': '47', 'district': '8', 'party': 'D'},
 {'name': 'Joanne Brekke', 'voter_id': '48', 'district': '32', 'party': 'D'},
 {'name': 'Art Broback', 'voter_id': '50', 'district': '28', 'party': 'R'},
 {'name': 'Jean-Marie Brough', 'voter_id': '54', 'district': '30', 'party': 'R'},
 {'name': 'Bill Brumsickle', 'voter_id': '63', 'district': '20', 'party': 'R'},
 {'name': 'Maria Cantwell', 'voter_id': '68', 'district': '44', 'party': 'D'},
 {'name': 'Gary Chandler', 'voter_id': '75', 'district': '13', 'party': 'R'},
 {'name': 'David Cooper', 'voter_id': '93', 'district': '18', 'party': 'D'},
 {'name': 'Bill Day', 'voter_id': '103', 'district': '3', 'party': 'D'},
 {'name': 'Dennis Dellwo', 'voter_id': '110', 'district': '3', 'party': 'D'},
 {'name': 'Randy Dorn', 'voter_id': '116', 'district': '2', 'party': 'D'},
 {'name': 'Brian Ebersole', 'voter_id': '121', 'district': '29', 'party': 'D'}, 
 {'name': 'Betty Edmondson', 'voter_id': '123', 'district': '14', 'party': 'R'},
 {'name': 'Roy Ferguson', 'voter_id': '132', 'district': '48', 'party': 'R'},
 {'name': 'Greg Fisher', 'voter_id': '135', 'district': '33', 'party': 'D'},
 {'name': 'Elmira Forner', 'voter_id': '140', 'district': '47', 'party': 'R'}, 
 {'name': 'Steve Fuhrman', 'voter_id': '146', 'district': '7', 'party': 'R'},
 {'name': 'Lorraine Hine', 'voter_id': '186', 'district': '33', 'party': 'D'},
 {'name': 'Bruce Holland', 'voter_id': '192', 'district': '47', 'party': 'R'},
 {'name': 'Jay Inslee', 'voter_id': '205', 'district': '14', 'party': 'D'},
 {'name': 'Rob Johnson', 'voter_id': '214', 'district': '40', 'party': 'D'},
 {'name': 'Evan Jones', 'voter_id': '219', 'district': '24', 'party': 'D'},
 {'name': 'Joseph King', 'voter_id': '226', 'district': '49', 'party': 'D'},
 {'name': 'Richard King', 'voter_id': '227', 'district': '38', 'party': 'D'},
 {'name': 'Pete Kremen', 'voter_id': '234', 'district': '42', 'party': 'D'},
 {'name': 'June Leonard', 'voter_id': '245', 'district': '11', 'party': 'D'},
 {'name': 'Gary Locke', 'voter_id': '251', 'district': '37', 'party': 'D'},
 {'name': 'Curtis Ludwig', 'voter_id': '254', 'district': '8', 'party': 'D'},
 {'name': 'Fred May', 'voter_id': '264', 'district': '41', 'party': 'R'},
 {'name': 'Alex McLean', 'voter_id': '276', 'district': '12', 'party': 'R'},
 {'name': 'Holly Meyers', 'voter_id': '282', 'district': '17', 'party': 'D'},
 {'name': 'Todd Mielke', 'voter_id': '284', 'district': '5', 'party': 'R'},
 {'name': 'Todd Mielke', 'voter_id': '284', 'district': '6', 'party': 'R'},
 {'name': 'Louise Miller', 'voter_id': '285', 'district': '45', 'party': 'R'},
 {'name': 'John Moyer', 'voter_id': '297', 'district': '6', 'party': 'R'},
 {'name': 'John Moyer', 'voter_id': '297', 'district': '3', 'party': 'R'},
 {'name': 'Harvey Myers', 'voter_id': '301', 'district': '15', 'party': 'R'},
 {'name': 'Darwin Nealey', 'voter_id': '304', 'district': '9', 'party': 'R'},
 {'name': "John O'Brien", 'voter_id': '311', 'district': '37', 'party': 'D'},
 {'name': 'George Orr', 'voter_id': '320', 'district': '4', 'party': 'D'},
 {'name': 'George-Marshall Paris', 'voter_id': '325', 'district': '44', 'party': 'R'},
 {'name': 'W-Kim Peery', 'voter_id': '328', 'district': '17', 'party': 'D'},
 {'name': 'Larry Phillips', 'voter_id': '332', 'district': '36', 'party': 'D'},
 {'name': 'Wes Pruitt', 'voter_id': '341', 'district': '26', 'party': 'D'},
 {'name': 'Margaret Rayburn', 'voter_id': '349', 'district': '15', 'party': 'D'},
 {'name': 'Mike Riley', 'voter_id': '359', 'district': '19', 'party': 'D'},
 {'name': 'Judith Roland', 'voter_id': '363', 'district': '31', 'party': 'D'},
 {'name': 'Nancy Rust', 'voter_id': '367', 'district': '1', 'party': 'D'},
 {'name': 'Karen Schmidt', 'voter_id': '376', 'district': '23', 'party': 'R'},
 {'name': 'B-Jean Silver', 'voter_id': '369', 'district': '5', 'party': 'R'},
 {'name': 'B-Jean Silver', 'voter_id': '369', 'district': '6', 'party': 'R'},
 {'name': 'Duane Sommers, ', 'voter_id': '406', 'district': '6', 'party': 'R'},
 {'name': 'Art Sprenkle', 'voter_id': '410', 'district': '39', 'party': 'D'},
 {'name': 'Randy Tate', 'voter_id': '429', 'district': '25', 'party': 'R'},
 {'name': 'Christopher Vance', 'voter_id': '444', 'district': '31', 'party': 'R'}, 
 {'name': 'Art Wang', 'voter_id': '449', 'district': '27', 'party': 'D'},
 {'name': 'Simeon Wilson', 'voter_id': '472', 'district': '10', 'party': 'R'},
 {'name': 'Jesse Wineberry', 'voter_id': '474', 'district': '43', 'party': 'D'},
 {'name': 'Jesse Wineberry', 'voter_id': '474', 'district': '37', 'party': 'D'},
 {'name': 'John Wynne', 'voter_id': '481', 'district': '39', 'party': 'R'},
 {'name': 'Neil Amondson', 'voter_id': '5', 'district': '20', 'party': 'R'},
 {'name': 'Cliff Bailey', 'voter_id': '15', 'district': '39', 'party': 'R'},
 {'name': 'Scott Barr', 'voter_id': '19', 'district': '7', 'party': 'R'},
 {'name': 'Albert Bauer', 'voter_id': '25', 'district': '49', 'party': 'D'},
 {'name': 'Alan Bluechel', 'voter_id': '42', 'district': '45', 'party': 'R'},
 {'name': 'Emilio Cantu', 'voter_id': '67', 'district': '41', 'party': 'R'},
 {'name': 'Paul Conner', 'voter_id': '90', 'district': '24', 'party': 'D'},
 {'name': 'Ellen Craswell', 'voter_id': '95', 'district': '23', 'party': 'R'},
 {'name': 'Time Erwin', 'voter_id': '127', 'district': '44', 'party': 'R'},
 {'name': 'Marcus Gaspard', 'voter_id': '148', 'district': '25', 'party': 'D'},
 {'name': 'Frank Hansen', 'voter_id': '168', 'district': '13', 'party': 'D'},
 {'name': 'Jeannette Hayner', 'voter_id': '176', 'district': '16', 'party': 'R'},
 {'name': 'Jim Jesernig', 'voter_id': '211', 'district': '8', 'party': 'D'},
 {'name': 'Lela Kreidler,', 'voter_id': '232', 'district': '22', 'party': 'D'},
 {'name': 'Ken Madsen', 'voter_id': '256', 'district': '2', 'party': 'D'},
 {'name': 'Jim Matson', 'voter_id': '261', 'district': '14', 'party': 'R'},
 {'name': 'Patrick McMullen', 'voter_id': '278', 'district': '40', 'party': 'D'},
 {'name': 'Jack Metcalf', 'voter_id': '281', 'district': '10', 'party': 'R'},
 {'name': 'Ray Moore', 'voter_id': '290', 'district': '36', 'party': 'D'},
 {'name': 'Patty Murray', 'voter_id': '299', 'district': '1', 'party': 'D'},
 {'name': 'Gary Nelson', 'voter_id': '308', 'district': '21', 'party': 'R'},
 {'name': 'Janice Niemi', 'voter_id': '310', 'district': '43', 'party': 'D'},
 {'name': 'E-G Patterson', 'voter_id': '326', 'district': '9', 'party': 'R'},
 {'name': 'Dwight Pelz', 'voter_id': '329', 'district': '37', 'party': 'D'},
 {'name': 'A-L Rasmussen', 'voter_id': '345', 'district': '29', 'party': 'D'},
 {'name': 'Nina Rinehart', 'voter_id': '360', 'district': '46', 'party': 'D'},
 {'name': 'Gerald Saling', 'voter_id': '370', 'district': '5', 'party': 'R'},
 {'name': 'George Sellar', 'voter_id': '387', 'district': '12', 'party': 'R'},
 {'name': 'Sylvia Skratek', 'voter_id': '399', 'district': '47', 'party': 'D'},
 {'name': 'Adam Smith', 'voter_id': '402', 'district': '33', 'party': 'D'},
 {'name': 'Linda Smith', 'voter_id': '403', 'district': '18', 'party': 'R'},
 {'name': 'Lois Stratton', 'voter_id': '414', 'district': '3', 'party': 'D'},
 {'name': 'Dean Sutherland', 'voter_id': '423', 'district': '17', 'party': 'D'},
 {'name': 'Phil Talmadge', 'voter_id': '428', 'district': '34', 'party': 'D'},
 {'name': 'Leo Thorsness', 'voter_id': '434', 'district': '11', 'party': 'R'},
 {'name': 'Larry Vognild', 'voter_id': '445', 'district': '38', 'party': 'D'},
 {'name': 'Peter von-Reichbauer', 'voter_id': '446', 'district': '30', 'party': 'R'},
 {'name': 'Alvin Williams', 'voter_id': '467', 'district': '32', 'party': 'D'},
 {'name': 'R-Lorraine Wojahn', 'voter_id': '477', 'district': '27', 'party': 'D'},
 {'name': 'Myron Kreidler', 'voter_id': '233', 'district': '22', 'party': 'D'},
 {'name': 'Wanda Hansen', 'voter_id': '169', 'district': '13', 'party': 'D'},
 {'name': 'Susan Sumner', 'voter_id': '419', 'district': '28', 'party': 'R'},
 {'name': 'Barbara Holm', 'voter_id': '639', 'district': '35', 'party': 'D'},
 {'name': 'Kathleen Drew', 'voter_id': '570', 'district': '5', 'party': 'D'},
 {'name': 'Valoria Loveland', 'voter_id': '562', 'district': '16', 'party': 'D'},
 {'name': 'Kevin Quigley', 'voter_id': '563', 'district': '39', 'party': 'D'},
 {'name': 'David Chappell', 'voter_id': '621', 'district': '20', 'party': 'D'},
 {'name': 'Barbara Cothern', 'voter_id': '629', 'district': '1', 'party': 'D'},
 {'name': 'Stanley Flemming', 'voter_id': '636', 'district': '28', 'party': 'D'},
 {'name': 'Dale Foreman', 'voter_id': '637', 'district': '12', 'party': 'R'},
 {'name': 'Mick Hansen', 'voter_id': '638', 'district': '13', 'party': 'D'},
 {'name': 'Jim Johanson', 'voter_id': '640', 'district': '44', 'party': 'D'},
 {'name': 'Linda Johnson', 'voter_id': '641', 'district': '1', 'party': 'D'},
 {'name': 'Sue Karahalios', 'voter_id': '642', 'district': '10', 'party': 'D'},
 {'name': 'Dave Lemmon', 'voter_id': '645', 'district': '14', 'party': 'D'},
 {'name': 'Jim Springer', 'voter_id': '656', 'district': '18', 'party': 'D'},
 {'name': 'Brian Thomas', 'voter_id': '660', 'district': '5', 'party': 'R'},
 {'name': 'Cathy Wolfe', 'voter_id': '662', 'district': '22', 'party': 'D'},
 {'name': 'Bill Backlund', 'voter_id': '1369', 'district': '45', 'party': 'R'},
 {'name': 'Vivian Caver', 'voter_id': '1408', 'district': '37', 'party': 'D'},
 {'name': 'Thomas Moak', 'voter_id': '1351', 'district': '8', 'party': 'D'},
 {'name': 'Ray Schow', 'voter_id': '1352', 'district': '30', 'party': 'R'},
 {'name': 'Barney Beeksma', 'voter_id': '1582', 'district': '10', 'party': 'R'},
 {'name': 'Jerry Blanton', 'voter_id': '1569', 'district': '21', 'party': 'R'},
 {'name': 'Gene Goldsmith', 'voter_id': '1575', 'district': '42', 'party': 'R'},
 {'name': 'Steve Hargrove', 'voter_id': '1588', 'district': '23', 'party': 'R'},
 {'name': 'Tomm Huff', 'voter_id': '1578', 'district': '26', 'party': 'R'},
 {'name': 'Kathy Lambert', 'voter_id': '1589', 'district': '45', 'party': 'R'},
 {'name': 'John Pennington', 'voter_id': '1591', 'district': '18', 'party': 'R'},
 {'name': 'Renee Radcliff', 'voter_id': '1579', 'district': '21', 'party': 'R'},
 {'name': 'Mike Sherstad', 'voter_id': '1593', 'district': '1', 'party': 'R'},
 {'name': 'Darlene Fairley', 'voter_id': '1603', 'district': '32', 'party': 'D'},
 {'name': 'Hal Palmer', 'voter_id': '1667', 'district': '18', 'party': 'R'},
 {'name': 'Calvin Goings', 'voter_id': '1938', 'district': '25', 'party': 'D'},
 {'name': 'Carl Scheuerman', 'voter_id': '1949', 'district': '29', 'party': 'D'},
 {'name': 'Brian Sullivan', 'voter_id': '2132', 'district': '29', 'party': 'D'},
 {'name': 'Mike Wensman', 'voter_id': '2222', 'district': '41', 'party': 'R'},
 {'name': 'Lena Swanson', 'voter_id': '2202', 'district': '35', 'party': 'D'},
 {'name': 'Glenn Anderson', 'voter_id': '2147', 'district': '5', 'party': 'R'},
 {'name': 'John Blalock', 'voter_id': '2296', 'district': '33', 'party': 'D'},
 {'name': 'Carolyn Edmonds', 'voter_id': '3471', 'district': '32', 'party': 'D'},
 {'name': 'Michael Stensen', 'voter_id': '3485', 'district': '31', 'party': 'D'},
 {'name': 'Joe Marine', 'voter_id': '6307', 'district': '21', 'party': 'R'},
 {'name': 'John Wyss', 'voter_id': '11176', 'district': '6', 'party': 'R'},
 {'name': 'Eric Oemig', 'voter_id': '12067', 'district': '45', 'party': 'D'},
 {'name': 'Gina McCabe', 'voter_id': '20759', 'district': '14', 'party': 'R'},
 {'name': 'William Jenkin', 'voter_id': '26172', 'district': '16', 'party': 'R'},
 {'name': 'Keith Wagoner', 'voter_id': '28317', 'district': '39', 'party': 'R'}]

In [110]:
manual_df = pd.DataFrame(manual)

In [111]:
manual_df

,district,name,party,voter_id
0,42,Ann Anderson,R,7
1,19,Bob Basich,D,23
2,21,John Beck,R,32
3,22,Jennifer Belcher,D,34
4,45,John Betrozoff,R,40
5,20,Rose Bowman,R,45
6,42,Dennis Braddock,D,46
7,8,Lane Bray,D,47
8,32,Joanne Brekke,D,48
9,28,Art Broback,R,50


In [112]:
manual_df['first_name'] = manual_df['name'].apply(lambda x: x.split()[0])

In [113]:
manual_df['name'] = manual_df['name'].apply(lambda x: x.split()[1])

In [114]:
manual_df.head()

,district,name,party,voter_id,first_name
0,42,Anderson,R,7,Ann
1,19,Basich,D,23,Bob
2,21,Beck,R,32,John
3,22,Belcher,D,34,Jennifer
4,45,Betrozoff,R,40,John


In [115]:
MERGED = legislators_df_merged.merge(manual_df, how='outer', on='voter_id')

In [116]:
MERGED.head()

,biennium,voter_id,voter_name,voting_agency,c_district,c_id,c_first_name,c_party,district,name,party,first_name
0,1991-92,7,Anderson,House,NaN,NaN,NaN,NaN,42,Anderson,R,Ann
1,1993-94,7,Anderson,House,NaN,NaN,NaN,NaN,42,Anderson,R,Ann
2,1995-96,7,"Anderson, C.",Senate,NaN,NaN,NaN,NaN,42,Anderson,R,Ann
3,1991-92,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
4,1993-94,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN


In [117]:
def replace_null_with_first_name(x):
    if type(x['c_first_name']) == float:
        return x['first_name']
    else:
        return x['c_first_name']

In [118]:
MERGED['c_first_name'] = MERGED.apply(lambda x : replace_null_with_first_name(x),axis=1)

In [119]:
MERGED

,biennium,voter_id,voter_name,voting_agency,c_district,c_id,c_first_name,c_party,district,name,party,first_name
0,1991-92,7,Anderson,House,NaN,NaN,Ann,NaN,42,Anderson,R,Ann
1,1993-94,7,Anderson,House,NaN,NaN,Ann,NaN,42,Anderson,R,Ann
2,1995-96,7,"Anderson, C.",Senate,NaN,NaN,Ann,NaN,42,Anderson,R,Ann
3,1991-92,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
4,1993-94,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
5,1995-96,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
6,1997-98,11,Appelwick,House,46,11,Marlin,Democrat,NaN,NaN,NaN,NaN
7,1991-92,17,Ballard,House,12,17,Clyde,Republican,NaN,NaN,NaN,NaN
8,1993-94,17,Ballard,House,12,17,Clyde,Republican,NaN,NaN,NaN,NaN
9,1995-96,17,Ballard,House,12,17,Clyde,Republican,NaN,NaN,NaN,NaN


In [ ]:
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''}, 
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},
  {'name': '', 'voter_id': '', 
  'district': '', 'party': ''}, 
 {'name': '', 'voter_id': '', 
  'district': '', 'party': ''},